<a href="https://colab.research.google.com/github/ivanksinggih/single_machine_scheduling_BP_rule_2Opt/blob/main/code_BP_rule_2Opt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# A. Library import

import time

details_debug = False

A function to calculate the total completion times based on a job sequence.

In [2]:
# B. Objective function definition

def objective_value_calculation(sequence):

  printed_sequence = [x+1 for x in sequence]  # In the computation, indices start from 0, but because the actual job indices start from 1, we increase all values in the list with 1

  if details_debug == True:
    print("-----------------------------------------------------")
    print("[Objective value calculation function]")
    print("sequence: ", printed_sequence)

  completion_time_per_sequence_index = []
  total_completion_times = 0

  # Add the distance between customers (in route) into the objective_value
  for i in range(len(sequence)):
    if i==0:
      completion_time_per_sequence_index += [processing_time[sequence[i]]]
    else:
      completion_time_per_sequence_index += [completion_time_per_sequence_index[i-1] + processing_time[sequence[i]]]
    total_completion_times += completion_time_per_sequence_index[i]

  if details_debug == True:
    print("completion_time_per_sequence_index: ", completion_time_per_sequence_index)
    print("total_completion_times: ", total_completion_times)
    print("-----------------------------------------------------")
    print()

  return total_completion_times

Program initialization:

In [3]:
# C. Program initialization

# Input data

# Job               1,  2,  3,  4
processing_time = [ 7,  4,  3, 10]

# Select one set of method(s) (select only one set, and place a # sign in front of not selected methods to deactivate them)
selected_methods = ["binary", "SPT", "2Opt"]
# selected_methods = ["binary", "SPT"]
# selected_methods = ["binary", "2Opt"]
# selected_methods = ["SPT", "2Opt"]
# selected_methods = ["binary"]
# selected_methods = ["SPT"]
# selected_methods = ["2Opt"]

Method 1: Binary Programming

In [4]:
# D. Binary Programming solving using GUROBI

if "binary" in selected_methods:

  !pip install gurobipy
  import gurobipy as gp
  from gurobipy import GRB
  GUROBI_timelimit = 15*60  # in seconds
  GUROBI_presolve = 2  # 0 = Deactivated, 1 = Activated, 2 = Both

  print()
  print("-----------------------------------------------------")
  print("Method 1: Binary Programming")
  print()

  # Code template taken from https://github.com/Gurobi/modeling-examples/blob/master/milk_collection/milk_collection.ipynb

  # Generate model
  model = gp.Model('single_machine_scheduling.lp')

  # Variable definition
  variable_indices_sequence_index = [*range(1,len(processing_time)+1)]
  variable_indices_jobs = [*range(1,len(processing_time)+1)]

  x_var = model.addVars(variable_indices_sequence_index, variable_indices_jobs, vtype=GRB.BINARY, name='x')

  print("x_var: ", x_var)

  # Set objective function
  model.setObjective(gp.quicksum(processing_time[j-1] * (len(processing_time) + 1 - i) * x_var[i,j] for i in variable_indices_sequence_index for j in variable_indices_jobs), GRB.MINIMIZE)

  # Set constraint 2
  model.addConstrs((gp.quicksum(x_var[i,j] for i in variable_indices_sequence_index) == 1 for j in variable_indices_jobs),
                  name='constraint_2')

  # Set constraint 3
  model.addConstrs((gp.quicksum(x_var[i,j] for j in variable_indices_jobs) == 1 for i in variable_indices_sequence_index),
                  name='constraint_3')

  # Solve model
  presolve_list = list()
  if GUROBI_presolve == 2:
      presolve_list.append(0)
      presolve_list.append(1)
  else:
      presolve_list.append(GUROBI_presolve)
  for presolve_setting in presolve_list:
              
      if presolve_setting == 0:
          print("Run GUROBI without presolving process")
      else:
          print("Run GUROBI with presolving process")
      print()

      start_time = time.time()
      model.reset(0)  # Reset model: remove any solutions from previous runs
      model.setParam(GRB.Param.TimeLimit, GUROBI_timelimit)
      model.setParam(GRB.Param.Presolve, presolve_setting)
      model.optimize()
      end_time = time.time()

      # Print results
      print("Objective value = ", model.ObjVal)

      for v in model.getVars():
          if v.x > 0:  # Print values that are larger than 0
              print('%s: %g' % (v.varName, v.x))

      print("Computational time (CPU time) = ", end_time - start_time, "seconds")
  print("-----------------------------------------------------")


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 12.8 MB 5.1 MB/s 

-----------------------------------------------------
Method 1: Binary Programming

Restricted license - for non-production use only - expires 2024-10-28
x_var:  {(1, 1): <gurobi.Var *Awaiting Model Update*>, (1, 2): <gurobi.Var *Awaiting Model Update*>, (1, 3): <gurobi.Var *Awaiting Model Update*>, (1, 4): <gurobi.Var *Awaiting Model Update*>, (2, 1): <gurobi.Var *Awaiting Model Update*>, (2, 2): <gurobi.Var *Awaiting Model Update*>, (2, 3): <gurobi.Var *Awaiting Model Update*>, (2, 4): <gurobi.Var *Awaiting Model Update*>, (3, 1): <gurobi.Var *Awaiting Model Update*>, (3, 2): <gurobi.Var *Awaiting Model Update*>, (3, 3): <gurobi.Var *Awaiting Model Update*>, (3, 4): <gurobi.Var *Awaiting Model Update*>, (4, 1): <gurobi.Var *Awaiting Model Update*>, (4, 2): <gurobi.Var *Awaiting Model Update*>, (4, 3): <gurobi.Var *Awaiting Model

Method 2: Shortest Processing Time Rule

In [5]:
# E. Shortest Processing Time rule

if "SPT" in selected_methods:

  print()
  print("-----------------------------------------------------")
  print("Method 2: Shortest Processing Time Rule")
  print()

  sequence = list()
  temp_processing_time = processing_time.copy()

  for i in range(len(processing_time)):  # Iteration to ensure finding n minimum values (n = number of jobs)
    print("Number of selected jobs: ", len(sequence))
    min_value = 999999
    selected_index = -1
    for j in range(len(processing_time)):  # Iteration to find a single minimum value among unselected jobs
      if temp_processing_time[j] < min_value:
        selected_index = j
        min_value = temp_processing_time[j]
    print("Available processing time to select: ", temp_processing_time, " (999999 represent the time for a selected job)")
    print("Selected index: ", selected_index)
    temp_processing_time[selected_index] = 999999
    sequence += [selected_index]
    print()

  objective = objective_value_calculation(sequence)
  sequence = [x+1 for x in sequence]  # In the computation, indices start from 0, but because the actual job indices start from 1, we increase all values in the list with 1
  
  print("Final solution for Method 2 (Shortest Processing Time Rule):")
  print("sequence: ", sequence)
  print("total_completion_times: ", objective)
  print("-----------------------------------------------------")



-----------------------------------------------------
Method 2: Shortest Processing Time Rule

Number of selected jobs:  0
Available processing time to select:  [7, 4, 3, 10]  (999999 represent the time for a selected job)
Selected index:  2

Number of selected jobs:  1
Available processing time to select:  [7, 4, 999999, 10]  (999999 represent the time for a selected job)
Selected index:  1

Number of selected jobs:  2
Available processing time to select:  [7, 999999, 999999, 10]  (999999 represent the time for a selected job)
Selected index:  0

Number of selected jobs:  3
Available processing time to select:  [999999, 999999, 999999, 10]  (999999 represent the time for a selected job)
Selected index:  3

Final solution for Method 2 (Shortest Processing Time Rule):
sequence:  [3, 2, 1, 4]
total_completion_times:  48
-----------------------------------------------------


Method 3: 2-Opt Algorithm

In [6]:
if "2Opt" in selected_methods:

  # F. 2-Opt algorithm

  print()
  print("-----------------------------------------------------")
  print("Method 3: 2-Opt Algorithm")
  print()

  # Given an initial sequence
  current_sequence = [4, 2, 1, 3]
  current_sequence = [x-1 for x in current_sequence]  # To adjust the indices with the ones used in the code (starting from 0)
  old_best_sequence = current_sequence.copy()
  old_min_value = objective_value_calculation(old_best_sequence)
  iteration = 1

  while (True):
    print("==============================================================")
    print("Iteration: ", iteration)
    print()
    
    new_min_value = 999999
    new_best_sequence = list()
    for i in range(len(processing_time)):  # Iteration to select the first index in the sequence for the exchange
      for j in range(len(processing_time)):  # Iteration to select the second index in the sequence for the exchange
        if j>i:
          Two_Opt_sequence = old_best_sequence.copy()
          temp = Two_Opt_sequence[i]
          Two_Opt_sequence[i] = Two_Opt_sequence[j]
          Two_Opt_sequence[j] = temp
          objective = objective_value_calculation(Two_Opt_sequence)
          printed_Two_Opt_sequence = [x+1 for x in Two_Opt_sequence]  # In the computation, indices start from 0, but because the actual job indices start from 1, we increase all values in the list with 1
          print("sequence: ", printed_Two_Opt_sequence, "; objective: ", objective)
          if objective < new_min_value:
            new_min_value = objective
            new_best_sequence = Two_Opt_sequence.copy()
    print()
    if old_min_value <= new_min_value:
      break
    else:
      old_min_value = new_min_value
      old_best_sequence = new_best_sequence.copy()
      printed_old_best_sequence = [x+1 for x in old_best_sequence]  # In the computation, indices start from 0, but because the actual job indices start from 1, we increase all values in the list with 1
      print("New best sequence --> ", printed_old_best_sequence, "; objective: ", old_min_value)
      print()

    iteration += 1

  objective = objective_value_calculation(old_best_sequence)
  old_best_sequence = [x+1 for x in old_best_sequence]  # In the computation, indices start from 0, but because the actual job indices start from 1, we increase all values in the list with 1

  print("Final solution for Method 3 (2-Opt Algorithm):")
  print("sequence: ", old_best_sequence)
  print("total_completion_times: ", objective)
  print("-----------------------------------------------------")


-----------------------------------------------------
Method 3: 2-Opt Algorithm

Iteration:  1

sequence:  [2, 4, 1, 3] ; objective:  63
sequence:  [1, 2, 4, 3] ; objective:  63
sequence:  [3, 2, 1, 4] ; objective:  48
sequence:  [4, 1, 2, 3] ; objective:  72
sequence:  [4, 3, 1, 2] ; objective:  67
sequence:  [4, 2, 3, 1] ; objective:  65

New best sequence -->  [3, 2, 1, 4] ; objective:  48

Iteration:  2

sequence:  [2, 3, 1, 4] ; objective:  49
sequence:  [1, 2, 3, 4] ; objective:  56
sequence:  [4, 2, 1, 3] ; objective:  69
sequence:  [3, 1, 2, 4] ; objective:  51
sequence:  [3, 4, 1, 2] ; objective:  60
sequence:  [3, 2, 4, 1] ; objective:  51

Final solution for Method 3 (2-Opt Algorithm):
sequence:  [3, 2, 1, 4]
total_completion_times:  48
-----------------------------------------------------
